In [4]:
import rdimacs as rd

filename = 'path_to_dimacs_cnf_file.cnf'
num_vars, clauses = rd.read_dimacs(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_dimacs_cnf_file.cnf'

In [ ]:
def animate_dpll(num_vars, clauses):
    fig, ax = plt.subplots()
    assignment = []

    def update(num):
        ax.clear()
        if num < len(assignment):
            current_assignment = assignment[:num]
        else:
            result = dpll(clauses, assignment)
            if result is False:
                plt.title('Unsatisfiable')
                return
            assignment.extend(result)
            current_assignment = assignment
        
        G = generate_interaction_graph(num_vars, clauses)
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
        plt.title(f'Step {num + 1}')
    
    ani = FuncAnimation(fig, update, frames=range(len(clauses)), repeat=False)
    plt.show()